In [56]:
!pip install streamlit

     |████████████████████████████████| 7.2MB 2.6MB/s 
     |████████████████████████████████| 4.4MB 40.9MB/s 
     |████████████████████████████████| 112kB 48.9MB/s 
     |████████████████████████████████| 102kB 9.5MB/s 
     |████████████████████████████████| 122kB 42.7MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp36-none-any.whl size=13450 sha256=2021954166c611f09c92a0c3fd3b7ab686f9a69dec7b9112b6b304fe29a62ae9
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=5d79ba981f67a11fc222e107c4c77c67bd687be7fd5e02d3a72190be57a217c9
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=ae1c05477de9acca1e05a3a23a7f97bf57238e714f3890bcd40cef5c628ea76d
  Stored in directory: /root/.cache/pip/wheel

In [17]:
pip install adversarial-robustness-toolbox

     |████████████████████████████████| 655kB 3.4MB/s 
     |████████████████████████████████| 6.8MB 17.3MB/s 
     |████████████████████████████████| 235kB 42.3MB/s 
     |████████████████████████████████| 20.8MB 60.4MB/s 
     |████████████████████████████████| 747kB 42.2MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [1]:
!pip install pyngrok

In [2]:
!wget https://raw.githubusercontent.com/TanmayKhot/Fluorine/master/imagenet_classes.txt

--2020-09-14 09:36:13--  https://raw.githubusercontent.com/TanmayKhot/Fluorine/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21675 (21K) [text/plain]
Saving to: ‘imagenet_classes.txt.1’

imagenet_classes.tx 100%[===================>]  21.17K  --.-KB/s    in 0.01s   

2020-09-14 09:36:13 (2.01 MB/s) - ‘imagenet_classes.txt.1’ saved [21675/21675]



In [46]:
%%writefile main.py
import streamlit as st
PAGE_CONFIG = {"page_title":"Image Classifier Visualizer with Adv Attacks.io", "page_icon":":smiley:", "layout":"centered"}
st.beta_set_page_config(**PAGE_CONFIG)
import numpy as np
import time
import torch
import torchvision.models as models
from torchvision import transforms as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
import urllib.request
from art.attacks.evasion import *
from art.estimators.classification import PyTorchClassifier
 
with open('imagenet_classes.txt') as f:
  classes = [line.strip() for line in f.readlines()]
st.set_option('deprecation.showfileUploaderEncoding', False)
html_temp = '''
    <div>
    <center><h1>Image Classifier</h1></center>
    <center><h2>with</h2></center>
    <center><h1>Adversarial Attack Visualizer</h1></center>
    <h1></h1>
    </div>
    '''
st.markdown(html_temp, unsafe_allow_html=True)
import base64
 
def get_base64_of_bin_file(bin_file):
    with open(bin_file, 'rb') as f:
        data = f.read()
    return base64.b64encode(data).decode()
 
def set_png_as_page_bg(jpg_file):
    bin_str = get_base64_of_bin_file(jpg_file)
    page_bg_img = '''
    <style>
    body {
    background-image: url("data:image/jpg;base64,%s");
    background-size: cover;
    }
    </style>
    ''' % bin_str
    st.markdown(page_bg_img, unsafe_allow_html=True)
    return
 
set_png_as_page_bg('nbgr.jpg')
 
def classify(model, image, classes, model_select, choice):
  model.eval()
  from torchvision import transforms
  
  preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])
  input_tensor = preprocess(image)
  input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
 
  # move the input and model to GPU for speed if available
  if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')
  with torch.no_grad():
    output = model(input_batch)
  _, index = torch.max(output, 1)
  percentage = torch.nn.functional.softmax(output, dim=1)[0] * 100
  
  
  if choice == 1:
    progress_bar = st.sidebar.progress(0)
    status_text = st.sidebar.empty()
    for i in range(1,101):
      status_text.text("Classifying")
      if i >= 30:
        status_text.text("Classifying.")
      if i >= 60:
        status_text.text("Classifying..")
      if i >= 90:
        status_text.text("Classifying...")
      progress_bar.progress(i)
      time.sleep(0.02)
    progress_bar.empty()
    time.sleep(1.5)
    status_text.empty()
  
  st.success('Hey! {} classified your image as a {} with confidence {}'.format(model_select, classes[index[0]], percentage[index[0]].item()))

#end of def classify 

def paginator(label, items, items_per_page=2, on_sidebar=True):
    max_index =  items_per_page
    import itertools
    return itertools.islice(enumerate(items), max_index)

#end of paginator

def adversarial(image, model, attack, attack_select, epsilon):
  width, height = image.size
  transform = T.Compose([            
    T.Resize(256),                    
    T.CenterCrop(224),                
    T.ToTensor(),                     
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
  ])
  
  transformed_img = transform(image)
  input_img = transformed_img.unsqueeze(0)
  advx = attack.generate(x=input_img)
  rescaled = (advx - advx.min()) / (advx.max() - advx.min())
  rescaled = (255*np.transpose(np.squeeze(rescaled))).astype(np.uint8)
  rescaled = np.moveaxis(rescaled ,0, 1)
  img = Image.fromarray(rescaled)
  f_trans = T.Compose([
                       T.Resize(256),
                       T.CenterCrop(224)])
  img_f = f_trans(image)
  images=[img_f, img]
  captions=['Uploaded Image', 'Image after Adversarial Attack']
  image_iter = paginator("",images)
  index, fimg = map(list, zip(*image_iter))
  st.image(fimg, width = 328, caption = captions)
  transformed_img = transform(img)
  final_img = transformed_img.unsqueeze(0)
  output = model(final_img)
  _, index = torch.max(output, 1)
  percentage = torch.nn.functional.softmax(output, dim=1)[0] * 100
  st.warning('Hey! {} classified your image as a {} with confidence {} after implementing {}'.format(model_select, classes[index[0]], percentage[index[0]].item(), attack_select))


select =  st.sidebar.selectbox('',('Home','Image Classifier','Image Classifier with Adversarial Attack Visualizer'))

if select == 'Image Classifier':
    model_select = st.sidebar.selectbox('Please select a Classification Model from the list',('','AlexNet', 'DenseNet', 'ResNet','VGG'))
    html_temp = '''
    <div>
    <center><h3>Please enter Image for Classification</h3></center>
    </div>
    '''
    st.markdown(html_temp, unsafe_allow_html=True)
    opt = st.selectbox("How do you want to upload the image for classification?\n", ('Please Select', 'Upload image via link', 'Upload image from device'))
 
    if opt == 'Upload image from device':
      file = st.file_uploader('Select', type = ['jpg', 'png', 'jpeg'])
      st.set_option('deprecation.showfileUploaderEncoding', False)
      if file is not None:
            image = Image.open(file)

    elif opt == 'Upload image via link':
        img = st.text_input('Enter the Image Address')
        if st.button('Submit'):
            image = Image.open(urllib.request.urlopen(img))

    try:
      if model_select == 'VGG':
        model = models.vgg16(pretrained=True)
        st.image(image, width = 300, caption = 'Uploaded Image')
        classify(model, image, classes, model_select, 1)
      elif model_select == 'AlexNet':
        model = models.alexnet(pretrained=True)
        st.image(image, width = 300, caption = 'Uploaded Image')
        classify(model, image, classes, model_select, 1)
      elif model_select == 'DenseNet':
        model = models.densenet121(pretrained=True)
        st.image(image, width = 300, caption = 'Uploaded Image')
        classify(model, image, classes, model_select, 1)
      elif model_select == 'ResNet':
        model = models.resnet101(pretrained=True)
        st.image(image, width = 300, caption = 'Uploaded Image')
        classify(model, image, classes, model_select, 1)
    except:
      pass      
 
    if opt == "Please Select":
        pass
 
if select == 'Image Classifier with Adversarial Attack Visualizer':    
    model_select = st.sidebar.selectbox('Please select a Classification Model from the list:',('','AlexNet', 'DenseNet', 'ResNet','VGG'))
    attack_select = st.sidebar.selectbox('Please select an Adversarial Attack from the list:',('','DeepFool','FGSM','AutoAttack','BasicIterativeMethod')) 
    epsilon = st.sidebar.number_input('Enter Epsilon value(e>0):')
    if epsilon < 0:
      warn = st.sidebar.error("Epsilon value should be a positive number. Please Re-enter...")
      time.sleep(3)
      warn.empty()
    html_temp = '''
    <div>
    <center><h3>Please enter Image for Classification</h3></center>
    </div>
    '''
    st.markdown(html_temp, unsafe_allow_html=True)
    opt = st.selectbox("How do you want to upload the image for classification?\n", ('Please Select', 'Upload image via link', 'Upload image from device'))
 
    if opt == 'Upload image from device':
      file = st.file_uploader('Select', type = ['jpg', 'png', 'jpeg'])
      st.set_option('deprecation.showfileUploaderEncoding', False)
      if file is not None:
            image = Image.open(file)

    elif opt == 'Upload image via link':
        img = st.text_input('Enter the Image Address')
        if st.button('Submit'):
            image = Image.open(urllib.request.urlopen(img))

    if model_select == 'VGG':
       model = models.vgg16(pretrained=True)
       
    elif model_select == 'AlexNet':
        model = models.alexnet(pretrained=True)
        
    elif model_select == 'DenseNet':
        model = models.densenet121(pretrained=True)
        
    elif model_select == 'ResNet':
        model = models.resnet101(pretrained=True)

    if model_select != '':
      criterion = nn.CrossEntropyLoss()
      optimizer = optim.Adam(model.parameters(), lr=0.01)
      classifier = PyTorchClassifier(
        model=model,
        clip_values=(0, 1),
        loss=criterion,
        optimizer=optimizer,
        input_shape=(3, 224, 224),
        nb_classes=1000,
      )
    try:
      if attack_select == 'DeepFool':
        attack = DeepFool(classifier, epsilon=epsilon)
        adversarial(image, model, attack,  attack_select, epsilon)
      elif attack_select == 'FGSM':
        attack = FastGradientMethod(estimator = classifier, eps = epsilon) 
        adversarial(image, model, attack,  attack_select, epsilon)
      elif attack_select == 'AutoAttack':
        attack = AutoAttack(estimator = classifier, eps = epsilon)
        adversarial(image, model, attack,  attack_select, epsilon)
      elif attack_select == 'BasicIterativeMethod':
        attack = BasicIterativeMethod(classifier, epsilon)
        adversarial(image, model, attack,  attack_select, epsilon)
    except:
      pass

    if opt == "Please Select":
        pass
 
if select == 'Home':
    
    html_temp = '''
    <div style = "background-color: rgba(25,25,112,0.20); padding: 20px; padding-bottom: 30px; padding-left: 20px; padding-right: 20px">
    <center><h2>Hey there!!! Welcome! &#x1F609;</h2></center>
    <h1></h1>
    <center><h3>Enter any image of your choice and our models will classify it for you.</h3></center>
    <center><h4>Wait.. There's more to it</h4></center>
    <center><h3>Select an Adversarial Attack of your choice to get wierdly exciting results!!!</h3></center>
    <h3></h3>
    <center><h4>To know more about Image Classifier with PyTorch click <a href = "https://towardsdatascience.com/how-to-train-an-image-classifier-in-pytorch-and-use-it-to-perform-basic-inference-on-single-images-99465a1e9bf5"  target="_blank">here</a>.</h4></center> 
    <center><h4>To know more about Adversarial Attacks click <a href = "https://towardsdatascience.com/breaking-neural-networks-with-adversarial-attacks-f4290a9a45aa"  target="_blank">here</a>.</h4></center>
     
    </div>
    '''
    st.markdown(html_temp, unsafe_allow_html=True)    

Overwriting main.py


In [4]:
!ngrok authtoken 1hQAE2uizes35CU6alr9SoHkWrn_5hci3LaKPin2vFJcvLhks

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [5]:
!streamlit run main.py&>/dev/null&
!pgrep streamlit

2069


In [7]:
from pyngrok import ngrok

In [8]:
public_url = ngrok.connect(port='8501')
public_url

'http://20afa8158f14.ngrok.io'

In [48]:
!streamlit config show

/bin/bash: streamlit: command not found


In [55]:
ngrok.kill()